In [1]:
%config IPCompleter.greedy=True
import linecache
import os
import time
import matplotlib.pyplot as plt
import pandas as pd

from clorm import *
from clorm import monkey
monkey.patch()  # must call this before importing clingo
from clorm.clingo import *
from clorm.clingo import Control

class Operation:
    def __init__(self, data_type, var, val, process_id, data_id):
        self.data_type = data_type
        self.var = var
        self.val = val
        self.process_id = process_id
        self.data_id = data_id
        
class Rd(Predicate):
    op_1 = StringField

class Wrt(Predicate):
    op_1 = StringField

class Po(Predicate):
    op_1 = StringField
    op_2 = StringField

class Sv(Predicate):
    op_1 = StringField
    op_2 = StringField

class Wr(Predicate):
    op_1 = StringField
    op_2 = StringField

class Initread(Predicate):
    init_op = StringField

class Co(Predicate):
    op_1 = StringField
    op_2 = StringField

class Reach(Predicate):
    op_1 = StringField
    op_2 = StringField
    
class Bad_CyclicCO(Predicate):
    op_1 = StringField

class Bad_WriteCOInitRead(Predicate):
    op_1 = StringField
    op_2 = StringField

In [2]:
def generate_dict(data_para):
    '''
    Return the dictionary grouped by the process_id.
    Not clarify whether the data_type is read or write.
    :param data_para: a list of input data. e.g. ['w(x,1,0,0)\n', 'r(x,2,0,1)\n', 'w(x,2,1,2)\n', 'r(x,1,1,3)\n']
    :return: the dictionary grouped by the process_id.
    '''
    dict = {}
    for temp in data_para:
        temp = temp.strip('\n')
        arr = temp[2:-1].split(',')
        op = Operation(temp[0], arr[0], arr[1], arr[2], arr[3])
        if arr[2] in dict:
            dict[arr[2]].append(op)
        else:
            dict[arr[2]] = [op]
    return dict


def fact_formatter(fact_type, op1, op2):
    '''
    Format the output string of relation between two operations.
    :param fact_type: 'wr'/'sv'/'po'
    :param op1: operation1
    :param op2: operation2
    :return:
    '''
    return fact_type + '("' + op1.data_type + '(' + op1.var + ',' + op1.val + ',id' + op1.data_id + ')","' + op2.data_type + '(' + op2.var + ',' + op2.val + ',id' + op2.data_id + ')").\n'


def write_facts(dict):
    '''
    1. Iterate operations by each process. Choose an operation.
    2. Add 'wrt' or 'rd' fact.
    3. Calculate 'po', 'sv' and 'wr' relationships with other operations in the same process.
    4. Calculate 'sv' and 'wr' with operations in subsequent processes.
    Note that 'wr' fact contains both w-r and r-w relations.
    :param dict: dictionary returned by generate_dict()
    :param f: file writer
    :return: None
    '''
    keys = list(dict.keys())
    i = 0
    facts = []
    while i < len(dict.keys()):
        ops_len = len(dict[keys[i]])
        for temp_op in range(ops_len):
            if dict[keys[i]][temp_op].data_type == 'r':
                if dict[keys[i]][temp_op].val == '0':
                    facts.append('initread("r(' + dict[keys[i]][temp_op].var + ',' + dict[keys[i]][temp_op].val + ',id' +
                            dict[keys[i]][
                                temp_op].data_id + ')").\n')
                else:
                    facts.append('rd("r(' + dict[keys[i]][temp_op].var + ',' + dict[keys[i]][temp_op].val + ',id' +
                            dict[keys[i]][
                                temp_op].data_id + ')").\n')
            else:
                facts.append('wrt("w(' + dict[keys[i]][temp_op].var + ',' + dict[keys[i]][temp_op].val + ',id' +
                        dict[keys[i]][
                            temp_op].data_id + ')").\n')
            for index in range(temp_op + 1, ops_len):
                if int(dict[keys[i]][temp_op].data_id) < int(dict[keys[i]][index].data_id):
                    facts.append(fact_formatter('po', dict[keys[i]][temp_op], dict[keys[i]][index]))
                else:
                    facts.append(fact_formatter('po', dict[keys[i]][index], dict[keys[i]][temp_op]))
                if dict[keys[i]][temp_op].var == dict[keys[i]][index].var:
                    facts.append(fact_formatter('sv', dict[keys[i]][temp_op], dict[keys[i]][index]))
                    facts.append(fact_formatter('sv', dict[keys[i]][index], dict[keys[i]][temp_op]))
                    if dict[keys[i]][temp_op].data_type == 'r' and dict[keys[i]][index].data_type == 'w' and \
                            dict[keys[i]][temp_op].val == dict[keys[i]][index].val:
                        facts.append(fact_formatter('wr', dict[keys[i]][index], dict[keys[i]][temp_op]))
                    elif dict[keys[i]][temp_op].data_type == 'w' and dict[keys[i]][index].data_type == 'r' and \
                            dict[keys[i]][temp_op].val == dict[keys[i]][
                        index].val:
                        facts.append(fact_formatter('wr', dict[keys[i]][temp_op], dict[keys[i]][index]))
            if i != len(dict.keys()) - 1:
                for index in range(i + 1, len(keys)):
                    for op in dict[keys[index]]:
                        if dict[keys[i]][temp_op].var == op.var:
                            facts.append(fact_formatter('sv', dict[keys[i]][temp_op], op))
                            facts.append(fact_formatter('sv', op, dict[keys[i]][temp_op]))
                            if dict[keys[i]][temp_op].data_type == 'r' and op.data_type == 'w' and dict[keys[i]][
                                temp_op].val == op.val:
                                facts.append(fact_formatter('wr', op, dict[keys[i]][temp_op]))
                            elif dict[keys[i]][temp_op].data_type == 'w' and op.data_type == 'r' and dict[keys[i]][
                                temp_op].val == op.val:
                                facts.append(fact_formatter('wr', dict[keys[i]][temp_op], op))
        i += 1
    return facts

In [3]:
def store_facts(data):
    facts = FactBase()
    readZero = False
    for i in range (0,len(data)):
        if data[i].startswith('rd'):
            temp = data[i].split('"')
            temp1 = temp[1]
            facts.add(Rd(temp1))
        if data[i].startswith('wrt'):
            temp = data[i].split('"')
            temp1 = temp[1]
            facts.add(Wrt(temp1))
        if data[i].startswith('po'):
            temp = data[i].split('"')
            temp1 = temp[1]
            temp2 = temp[3]
            facts.add(Po(temp1,temp2))
        if data[i].startswith('sv'):
            temp = data[i].split('"')
            temp1 = temp[1]
            temp2 = temp[3]
            facts.add(Sv(temp1,temp2))
        if data[i].startswith('wr('):
            temp = data[i].split('"')
            temp1 = temp[1]
            temp2 = temp[3]
            facts.add(Wr(temp1,temp2))
        if data[i].startswith('initread'):
            temp = data[i].split('"')
            temp1 = temp[1]
            facts.add(Initread(temp1))
    return facts

In [4]:
def detection(url):
    ASP_PROGRAM = "clingo-version-3.lp"
    data = linecache.getlines(url)
    data_dict = generate_dict(data)
    facts_plain = write_facts(data_dict)
    facts = store_facts(facts_plain)
    
    time1 = time.time()
    ctrl = Control(unifier=[Bad_CyclicCO, Bad_WriteCOInitRead])
    ctrl.load(ASP_PROGRAM)
    ctrl.add_facts(facts)
    ctrl.ground([("base", [])])
    solution = None
    return_list = [0, 0, 0]

    def on_model(model):
        solution = model.facts(atoms=True)
#         print('\n\n==============================\n\n')
#         print("All: {} len: {}".format(solution,len(solution)))
        bad_1 = solution.select(Bad_CyclicCO).get()
        bad_2 = solution.select(Bad_WriteCOInitRead).get()
        if (len(bad_1)) > 0:
            return_list[0] = 1
            return_list[2] = 1
        if (len(bad_2)) > 0:
            return_list[1] = 1
            return_list[2] = 1

    ctrl.solve(on_model=on_model)
    #     print('return_list',return_list)
    time2 = time.time()
    duration = time2 - time1
    return return_list, duration

In [5]:
if __name__ == '__main__':
    print('Start Galera!')
    url = '../DATASET/Galera'
    folders = (fn for fn in os.listdir(url))
    for folder in folders:
        if folder.startswith('.'):
            continue
        duration = 0
        inner_duration = 0
        df = pd.DataFrame(
            [], [],
            ['bad_pattern1', 'bad_pattern2', 'violation'])
        files = (fn for fn in os.listdir(url + '/' + folder))
        for file in files:
            time1 = time.time()
            file_url = url + '/' + folder + '/' + file
            detect_list, in_duration = detection(file_url)
            df.loc[file] = detect_list
            time2 = time.time()
            inner_duration += in_duration
            duration += time2 - time1
        df.loc['Total'] = df.apply(lambda x: x.sum())
        df.to_csv('../data/results/version_2/Galera/' + folder + '.csv')
        print(folder + ': Total {:.4f}'.format(duration / 200) +
              ', Clingo {:.4f}'.format(inner_duration / 200))


Start Galera!
00018.txt [1, 1, 1]
00030.txt [1, 1, 1]
00024.txt [1, 1, 1]
00178.txt [1, 1, 1]
00144.txt [1, 1, 1]
00150.txt [1, 1, 1]
00187.txt [1, 1, 1]
00193.txt [1, 1, 1]
00192.txt [1, 1, 1]
00186.txt [1, 1, 1]
00151.txt [1, 1, 1]
00145.txt [1, 1, 1]
00179.txt [1, 1, 1]
00025.txt [1, 1, 1]
00031.txt [1, 1, 1]
00019.txt [1, 1, 1]
00027.txt [1, 1, 1]
00033.txt [1, 1, 1]
00153.txt [1, 1, 1]
00147.txt [1, 1, 1]
00190.txt [1, 1, 1]
00184.txt [1, 1, 1]
00185.txt [1, 1, 1]
00191.txt [1, 1, 1]
00146.txt [1, 1, 1]
00152.txt [1, 1, 1]
00032.txt [1, 1, 1]
00026.txt [1, 1, 1]
00022.txt [1, 1, 1]
00036.txt [1, 1, 1]
00156.txt [1, 1, 1]
00142.txt [1, 1, 1]
00195.txt [1, 1, 1]
00181.txt [1, 1, 1]
00180.txt [1, 1, 1]
00194.txt [1, 1, 1]
00143.txt [1, 1, 1]
00157.txt [1, 1, 1]
00037.txt [1, 1, 1]
00023.txt [1, 1, 1]
00035.txt [1, 1, 1]
00021.txt [1, 1, 1]
00009.txt [1, 1, 1]
00141.txt [1, 1, 1]
00155.txt [1, 1, 1]
00169.txt [1, 1, 1]
00182.txt [1, 1, 1]
00196.txt [1, 1, 1]
00197.txt [1, 1, 1]
00183.

KeyboardInterrupt: 

In [ ]:
# TEST BLOCK

# if __name__ == '__main__':
#     folder_name = '../data/example3.txt'
#     df = pd.DataFrame([], [], ['bad_pattern1', 'bad_pattern2', 'violation'])
#     detect_list, duration = detection(folder_name)
#     df.loc['facts_2.txt'] = detect_list
#     df.loc['Total'] = df.apply(lambda x: x.sum())
#     df.to_csv('../data/results/version_2/Cockreach/123.csv')
#     print(detect_list)
#     print(df)
